In [0]:
# Load business and review data into DataFrames
# Load business.json into a DataFrame with the multiline option
business_df = spark.read.option("multiline", "true").json("dbfs:/FileStore/tables/business.json")

# Load review.json into a DataFrame with the multiline option
review_df = spark.read.option("multiline", "true").json("dbfs:/FileStore/tables/review.json")


In [0]:
# Filter businesses by category and drop the 'stars' column
from pyspark.sql.functions import array_contains

def filter_by_category(df, category):
    return df.filter(array_contains(df.categories, category))

filtered_business_df = filter_by_category(business_df, "Mexican")
filtered_business_df = filtered_business_df.drop('stars')



In [0]:
# Join business and review DataFrames on the 'business_id' column
def join_business_review(business_df, review_df):
    return business_df.join(review_df, on="business_id", how="inner")

joined_df = join_business_review(filtered_business_df, review_df)


In [0]:
# Group and aggregate the joined DataFrame to compute average rating and review count
from pyspark.sql.functions import count, avg

def group_and_aggregate(df):
    return df.groupBy("business_id", "name", "city", "state").agg(
        avg("stars").alias("average_rating"), count("review_id").alias("review_count")
    )

aggregated_df = group_and_aggregate(joined_df)


In [0]:
# Sort the aggregated DataFrame by average rating and review count in descending order
from pyspark.sql.functions import desc

# Function to sort the DataFrame
def sort_by_rating_and_review_count(df):
    return df.sort(desc("average_rating"), desc("review_count"))

# Sort the DataFrame
sorted_df = sort_by_rating_and_review_count(aggregated_df)


In [0]:
# Display the top N restaurants based on average rating and review count
# Function to display the top-rated businesses
def display_top_businesses(df, n):
    df.show(n)

# Display the top 10 restaurants
display_top_businesses(sorted_df, 10)


+--------------------+------+-------------+-----+--------------+------------+
|         business_id|  name|         city|state|average_rating|review_count|
+--------------------+------+-------------+-----+--------------+------------+
|tnhfDv5Il8EaGSXZG...|Garaje|San Francisco|   CA|           4.0|           1|
+--------------------+------+-------------+-----+--------------+------------+



In [0]:
print(f"Number of records in business_df: {business_df.count()}")
print(f"Number of records in review_df: {review_df.count()}")
print(f"Number of records in filtered_business_df: {filtered_business_df.count()}")


Number of records in business_df: 1
Number of records in review_df: 1
Number of records in filtered_business_df: 1


In [0]:
print(f"Number of records in joined_df: {joined_df.count()}")


Number of records in joined_df: 1
